In [ ]:
%pylab inline
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data.dataloader as dataloader
import torch.optim as optim

from torch.utils.data import TensorDataset
from torch.autograd import Variable
from torchvision import transforms
from torchvision.datasets import MNIST
import matplotlib.pyplot as plt
import os
from skimage import io, transform 
import cv2 as cv
import sklearn
from sklearn import metrics


SEED = 1

# CUDA?
cuda = torch.cuda.is_available()

# For reproducibility
torch.manual_seed(SEED)

if cuda:
    torch.cuda.manual_seed(SEED)

Populating the interactive namespace from numpy and matplotlib


In [ ]:
# mount drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import os
os.chdir("/content/gdrive/My Drive")
!ls

 0-.png   4-.png   8-.png       Biometrics.ipynb   DL论文
 1-.png   5-.png   9-.png      'Colab Notebooks'   Sketch.ipynb
 2-.png   6-.png   abc.png      data		   Untitled0.ipynb
 3-.png   7-.png   biometrics  'DL Lab'


In [ ]:
train = MNIST('./data', train=True, download=True, transform=transforms.Compose([
    transforms.ToTensor(), # ToTensor does min-max normalization. 
]), )

test = MNIST('./data', train=False, download=True, transform=transforms.Compose([
    transforms.ToTensor(), # ToTensor does min-max normalization. 
]), )

# Create DataLoader
dataloader_args = dict(shuffle=True, batch_size=256,num_workers=4, pin_memory=True) if cuda else dict(shuffle=True, batch_size=64)
train_loader = dataloader.DataLoader(train, **dataloader_args)
test_loader = dataloader.DataLoader(test, **dataloader_args)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x_1 = F.relu(self.conv1(x))
        x = F.max_pool2d(x_1, 2, 2)
        x_2 = F.relu(self.conv2(x))
        x = F.max_pool2d(x_2, 2, 2)
        x = x.view(-1, 4*4*50)
        x_3 = F.relu(self.fc1(x))
        h = F.softmax(self.fc2(x_3), dim=1)
        return h, x_3, x_2, x_1       
             
      
    
model = Model()
if cuda:
    model.cuda() # CUDA!
optimizer = optim.Adam(model.parameters(), lr=1e-3)    

In [ ]:
EPOCHS = 15
losses = []

model.train()
for epoch in range(EPOCHS):
    for batch_idx, (data, target) in enumerate(train_loader):
        # Get Samples
        data, target = Variable(data), Variable(target)
        
        if cuda:
            data, target = data.cuda(), target.cuda()
        
        # Init
        optimizer.zero_grad()

        # Predict
        y_pred, _, _, _ = model(data) 

        # Calculate loss
        loss = F.cross_entropy(y_pred, target)
        losses.append(loss.cpu().data)
#         losses.append(loss.cpu().data[0])        
        # Backpropagation
        loss.backward()
        optimizer.step()
        
        
        # Display
        if batch_idx % 100 == 1:
            print('\r Train Epoch: {}/{} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch+1,
                EPOCHS,
                batch_idx * len(data), 
                len(train_loader.dataset),
                100. * batch_idx / len(train_loader), 
                loss.cpu().data), 
                end='')
    # Eval
    evaluate_x = Variable(test_loader.dataset.test_data.type_as(torch.FloatTensor()))
    evaluate_y = Variable(test_loader.dataset.test_labels)
    if cuda:
        evaluate_x, evaluate_y = evaluate_x.cuda(), evaluate_y.cuda()

    model.eval()
    output, _, _, _ = model(evaluate_x[:,None,...])
    pred = output.data.max(1)[1]
    d = pred.eq(evaluate_y.data).cpu()
    accuracy = d.sum().type(dtype=torch.float64)/d.size()[0]
    
    print('\r Train Epoch: {}/{} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\t Test Accuracy: {:.4f}%'.format(
        epoch+1,
        EPOCHS,
        len(train_loader.dataset), 
        len(train_loader.dataset),
        100. * batch_idx / len(train_loader), 
        loss.cpu().data,
        accuracy*100,
        end=''))

In [ ]:
# based on confidence

batch_size = 10000
all_size = 100000
iters = 100

stats = dict()
for i in range(10):
    stats[i] = 0    
    
avgs = torch.zeros(iters, 10, 28*28)

for kk in range(iters):
  print(kk)
  
  z = torch.rand(all_size, 1, 28, 28) #.cuda()  
  z.cuda()

  all_preds = []
  all_confs = []
  all_idx = torch.ones(all_size, dtype = torch.uint8)

  for k in range(0,all_size, batch_size):
      y_pred, _, _, _ = model(z[k:k+batch_size].cuda())
      
      # erasing the low confident ones!
      y_pred[y_pred <= 0.3]= 0
      indices = torch.ones(y_pred.size(0), dtype = torch.uint8)
      # indices[torch.mean(y_pred, dim =1)==0] = 0 
      all_idx[k:k+batch_size] = indices 
      conf, pred = y_pred[indices==1].data.max(1)#[1]

      #if (not torch.isnan(conf).sum()) & (not torch.isnan(pred).sum())  :
      all_preds.append(pred)
      all_confs.append(conf)
  
  all_preds = torch.cat(all_preds)
  all_confs = torch.cat(all_confs)    
  


# erasing the low confident ones!
  z = z[all_idx]
    
  for i in range(10):
    stats[i] += torch.sum(all_preds==i)
    a = torch.mean(z[all_preds==i] , dim=0) 
#     print(z[preds==i].shape)
    avgs[kk, i] = a.reshape(28*28)

  
  
  
  
  
print(stats)  
  
  
  
# plotting  
save_path = '/content/gdrive/My Drive'


# setting NANs to 0
avgs[avgs != avgs] = 0 

dd = torch.mean(avgs, dim=0)

#dd = dd - grand_mean
for kk in range(10):
  
  fig = plt.figure()
  a = dd[kk]
  a = a.view(-1,28)
  b,_,_,_ = model(a[None,None,...].cuda())

  #a = torch.nn.functional.log_softmax(a)# torch.nn.functional.softmax(a)
  conf, c = b.data.max(1) #[1]
  plt.title(f'gt: {str(kk)}  -  pred: {str(c.cpu().data[0].numpy())} - conf: {str(conf.cpu().data[0].numpy())} -  size-gt: {stats[kk]}')  
  plt.imshow(a) #, cmap = 'gray')
  fig.savefig(os.path.join(save_path, str(kk)+'-.png'))



In [ ]:
 

# Load data
# data = mnist_data()

# tt = data.targets[data.targets==9]
# dd = data.data[data.targets==9] 
# data.targets = tt
# data.data = dd
# data_loader = torch.utils.data.DataLoader(data, batch_size=100, shuffle=True, drop_last =True)
# # Num batches

# z = next(iter(test_loader)) 
z = test.data # 10K test images
z = z[:,None,...]
z = z.type(torch.FloatTensor)
z = (z - z.min())/(z.max() - z.min())



# noise = torch.rand(1, 1, 28, 28) #.cuda()
weight = .5


# frequency of noise predictions
f, axarr = plt.subplots(11, 3)
# f.set_figheight(1.4)
# f.set_figwidth(15)
f.set_figheight(15)
f.set_figwidth(15)




# fixed noise 
axarr[0,0].imshow(torch.torch.ones_like(torch.squeeze(z[0,...],0)))
axarr[0,0].axis('off')


# fixed noise 
axarr[0,1].imshow(torch.squeeze(z[0,...],0))
axarr[0,1].axis('off')



y_pred,_ ,_ ,_ = model(z.cuda())
preds = y_pred.data.max(1)[1].cpu()
# print(preds)
freq = np.histogram(preds.numpy(), bins=list(range(11))) 
y_pos = np.arange(len(freq[0]))
axarr[0,2].bar(y_pos, freq[0]/freq[0].sum())
axarr[0,2].set_yticks([0.5,1]) 


mis_class = []


for i in range(1,11):
  # pattern
  pattern =  io.imread(os.path.join(save_path, str(i-1) + '-.png'))
  pattern = pattern[35:35+217,113:330,:]
  pattern = transform.resize(pattern, (28, 28))
  pattern = torch.from_numpy(pattern)
  pattern = pattern.type(torch.FloatTensor)
  pattern = torch.mean(pattern, dim=2)

  axarr[i,0].imshow(pattern)
  axarr[i,0].axis('off')
  
  
  # pattern + noise
  z_new = (1-weight)*z + weight*pattern #/ (weight+1)  # noise + stim
#   z_new = (z_new - z_new.min()) / (z_new.min() - z_new.min())
#   print(z_new.min(),z_new.max())
  axarr[i,1].imshow(torch.squeeze(z_new[0,...],0))
  axarr[i,1].axis('off')
  
  
  # frequency of pattern + noise predictions  
  y_pred,_ ,_ ,_ = model(z_new.cuda())
  preds = y_pred.data.max(1)[1].cpu()
#   print(preds)  
  freq = np.histogram(preds.numpy(), bins=list(range(11))) 
  y_pos = np.arange(len(freq[0]))
  axarr[i,2].bar(y_pos, freq[0]/freq[0].sum())
  axarr[i,2].set_xticks(list(range(10))) 
  axarr[i,2].set_yticks([0.5,1]) 
  preds 
  
  f.subplots_adjust(hspace=0.06) #, wspace=0.0, right = 0.8)
  f.show()
  
  
  # targetted attack!
  error = (preds == i-1).sum().type(torch.FloatTensor)/preds.size(0)
  # untargetted attack!
  # error = (preds != test.targets).sum().type(torch.FloatTensor)/preds.size(0)

  print(f'misclassification rate: {error}')
  mis_class.append(error)
  
print(np.mean(mis_class))

In [ ]:
# load the zero image


# Load data

# # Num batches

z = test.data # 10K test images
z = z[:,None,...]
z = z.type(torch.FloatTensor)
z = (z - z.min())/(z.max() - z.min())



# noise = torch.rand(1, 1, 28, 28) #.cuda()
weight = .9


# frequency of noise predictions
f, axarr = plt.subplots(11, 3)
# f.set_figheight(1.4)
# f.set_figwidth(15)
f.set_figheight(15)
f.set_figwidth(15)




# fixed noise 
axarr[0,0].imshow(torch.torch.ones_like(torch.squeeze(z[0,...],0)))
axarr[0,0].axis('off')


# fixed noise 
axarr[0,1].imshow(torch.squeeze(z[0,...],0))
axarr[0,1].axis('off')



y_pred, _, _, _= model(z.cuda())
preds = y_pred.data.max(1)[1].cpu()
# print(preds)
freq = np.histogram(preds.numpy(), bins=list(range(11))) 
y_pos = np.arange(len(freq[0]))
axarr[0,2].bar(y_pos, freq[0]/freq[0].sum())
axarr[0,2].set_yticks([0.5,1]) 


mis_class = []


for i in range(1,11):
  # pattern
  pattern = io.imread(os.path.join(save_path, str(i-1) + '-.png'))
  pattern = pattern[35:35+217,113:330,:]
  pattern = transform.resize(pattern, (28, 28))
  pattern = torch.from_numpy(pattern)
  pattern = pattern.type(torch.FloatTensor)
  pattern = torch.mean(pattern, dim=2)
  # pattern = np.randn(())

  axarr[i,0].imshow(pattern)
  axarr[i,0].axis('off')
  
  
  # pattern + noise
  z_new = (1-weight)*z + weight*pattern #/ (weight+1)  # noise + stim
#   z_new = (z_new - z_new.min()) / (z_new.min() - z_new.min())
#   print(z_new.min(),z_new.max())
  axarr[i,1].imshow(torch.squeeze(z_new[0,...],0))
  axarr[i,1].axis('off')
  
  
  # frequency of pattern + noise predictions  
  y_pred,_,_,_ = model(z_new.cuda())
  preds = y_pred.data.max(1)[1].cpu()
#   print(preds)  
  freq = np.histogram(preds.numpy(), bins=list(range(11))) 
  y_pos = np.arange(len(freq[0]))
  axarr[i,2].bar(y_pos, freq[0]/freq[0].sum())
  axarr[i,2].set_xticks(list(range(10))) 
  axarr[i,2].set_yticks([0.5,1]) 
  preds 
  
  f.subplots_adjust(hspace=0.06)
  f.show()
  
  
  error = (preds != test.targets).sum().type(torch.FloatTensor)/preds.size(0)
  print(f'misclassification rate: {error}')
  mis_class.append(error)
  
print(np.mean(mis_class))

In [ ]:
# load the zero image
# objects = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
# save_path = 'drive/My Drive/classification_images/10way-weighted'


z = torch.rand(10000, 1, 28, 28) #.cuda()
weight = .9


# frequency of noise predictions
f, axarr = plt.subplots(11, 3)
# f.set_figheight(1.4)
# f.set_figwidth(15)
f.set_figheight(15)
f.set_figwidth(15)




# fixed noise 
axarr[0,0].imshow(torch.torch.ones_like(torch.squeeze(z[0,...],0)))
axarr[0,0].axis('off')


# fixed noise 
axarr[0,1].imshow(torch.squeeze(z[0,...],0))
axarr[0,1].axis('off')



y_pred,_ , _, _ = model(z.cuda())
preds = y_pred.data.max(1)[1].cpu()
# print(preds)
freq = np.histogram(preds.numpy(), bins=list(range(11))) 
y_pos = np.arange(len(freq[0]))
axarr[0,2].bar(y_pos, freq[0]/freq[0].sum())
axarr[0,2].set_yticks([0.5,1]) 
# axarr[0,2].set_xticks(list(range(9))) 
# axarr[0,2].set_ylim([0,1])
# axarr[0,2].axis('on')



# f, axarr = plt.subplots(10, 3, )
# f.set_figheight(15)
# f.set_figwidth(15)

mis_class = []

for i in range(1,11):
  # pattern
  pattern =  io.imread(os.path.join(save_path, str(i-1) + '-.png'))
  pattern = pattern[35:35+217,113:330,:]
  pattern = transform.resize(pattern, (28, 28))
  pattern = torch.from_numpy(pattern)
  pattern = pattern.type(torch.FloatTensor)
  pattern = torch.mean(pattern, dim=2)

  axarr[i,0].imshow(pattern)
  axarr[i,0].axis('off')
  
  

  
  # pattern + noise
  z_new = (1-weight)*z + weight*pattern #/ (weight+1)  # noise + stim
  # z_new = (z_new - z_new.min()) / (z_new.min() - z_new.min())
#   print(z_new.min(),z_new.max())
  axarr[i,1].imshow(torch.squeeze(z_new[0,...],0))
  axarr[i,1].axis('off')

  
  
  
  # frequency of pattern + noise predictions  
  y_pred,_,_,_ = model(z_new.cuda())
  preds = y_pred.data.max(1)[1].cpu()
#   print(preds)  
  freq = np.histogram(preds.numpy(), bins=list(range(11))) 
  y_pos = np.arange(len(freq[0]))
  axarr[i,2].bar(y_pos, freq[0]/freq[0].sum())
  axarr[i,2].set_xticks(list(range(10))) 
  axarr[i,2].set_yticks([0.5,1]) 

  
  f.subplots_adjust(hspace=0.06) #, wspace=0.0, right = 0.8)
  f.show()
  
#   axarr[i,2].axis('off')

  error = (preds != i-1).sum().type(torch.FloatTensor)/preds.size(0)
  print(f'misclassification rate: {error}')
  mis_class.append(error)
  
print(np.mean(mis_class))

In [ ]:
#Create mean images
# computing the average mnist images
f, axarr = plt.subplots(2, 5)
# f.set_figheight(1.4)
# f.set_figwidth(15)
f.set_figheight(5)
f.set_figwidth(15)
f.subplots_adjust(hspace=0.36) #, wspace=0.0, right = 0.8)


mean_imgs = []
for i in range(10):
  plt.figure()
  m = torch.mean(train.data[train.targets == i].type(torch.FloatTensor), dim=0)
  mean_imgs.append(m)
  axarr[i//5,i%5].imshow(m) 
  

In [ ]:
print(len(mean_imgs))

10


In [ ]:
#mean images


z = test.data # 10K test images
z = z[:,None,...]
z = z.type(torch.FloatTensor)
z = (z - z.min())/(z.max() - z.min())

weight = .9


# frequency of noise predictions
f, axarr = plt.subplots(11, 3)
# f.set_figheight(1.4)
# f.set_figwidth(15)
f.set_figheight(15)
f.set_figwidth(15)

save_path = '/content/gdrive/My Drive'


# fixed noise 
axarr[0,0].imshow(torch.torch.ones_like(torch.squeeze(z[0,...],0)))
axarr[0,0].axis('off')


# fixed noise 
axarr[0,1].imshow(torch.squeeze(z[0,...],0))
axarr[0,1].axis('off')



y_pred,_ , _, _ = model(z.cuda())
preds = y_pred.data.max(1)[1].cpu()
# print(preds)
freq = np.histogram(preds.numpy(), bins=list(range(11))) 
y_pos = np.arange(len(freq[0]))
axarr[0,2].bar(y_pos, freq[0]/freq[0].sum())
axarr[0,2].set_yticks([0.5,1]) 




# f, axarr = plt.subplots(10, 3, )
# f.set_figheight(15)
# f.set_figwidth(15)

mis_class = []

for i in range(1,11):
  # pattern
  pattern = mean_imgs[i-1]
  pattern = (pattern - pattern.min()) / (pattern.max() - pattern.min())

  axarr[i,0].imshow(pattern)
  axarr[i,0].axis('off')
  
  

  
  # pattern + noise
  z_new = (1-weight)*z + weight*pattern #/ (weight+1)  # noise + stim
#   # z_new = (z_new - z_new.min()) / (z_new.min() - z_new.min())
# #   print(z_new.min(),z_new.max())
  axarr[i,1].imshow(torch.squeeze(z_new[0,...],0))
  axarr[i,1].axis('off')
  
  
  
  
  # frequency of pattern + noise predictions  
  y_pred, _, _, _ = model(z_new.cuda())
  preds = y_pred.data.max(1)[1].cpu()
#   print(preds)  
  freq = np.histogram(preds.numpy(), bins=list(range(11))) 
  y_pos = np.arange(len(freq[0]))
  axarr[i,2].bar(y_pos, freq[0]/freq[0].sum())
  axarr[i,2].set_xticks(list(range(10))) 
  axarr[i,2].set_yticks([0.5,1]) 

  
  f.subplots_adjust(hspace=0.06) #, wspace=0.0, right = 0.8)
  f.show()
  
#   axarr[i,2].axis('off')

  error = (preds != i-1).sum().type(torch.FloatTensor)/preds.size(0)
  print(f'misclassification rate: {error}')
  mis_class.append(error)
  
print(np.mean(mis_class))

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels
import copy
import matplotlib.cm as cm

x = np.arange(11)
ys = [i+x+(i*x)**2 for i in range(11)]
colors = cm.rainbow(np.linspace(0, 1, len(ys)))
# for y, c in zip(ys, colors):
#     plt.scatter(x, y, color=c)





# - measure response to a noise pattern
# - increase the bias for all neurons proportional to their response in a certain layer
# - see if the classification has been biased toward the class of favor

test_x = test.data.type(torch.FloatTensor).clone() # Variable(test_loader.dataset.test_data.type_as(torch.FloatTensor()))
test_y = test.targets.type(torch.ByteTensor).clone() #Variable(test_loader.dataset.test_labels)
# test_x = test_x/256

train_x = train.data.type(torch.FloatTensor).clone() # Variable(test_loader.dataset.test_data.type_as(torch.FloatTensor()))
train_y = train.targets.type(torch.ByteTensor).clone() #Variable(test_loader.dataset.test_labels)


f, axarr = plt.subplots(3, 5, )
f.set_figheight(20)
f.set_figwidth(30)


f2, axarr2 = plt.subplots(1, 21, )
f2.set_figheight(5)
f2.set_figwidth(30)

all_size = test_x.size(0)

layers = ['fc1', 'conv2', 'conv1']


dig_names = [str(i) for i in range(5,10)]

for l_idx, layer in enumerate(layers):

  for which in range(5,10):
    which_data = train_x[train_y == which]/255.    # 1 vs. 7

    with torch.no_grad():

      for k_idx, k in enumerate(range(0,11)):
        fraction_pred_favor = []

        # modulate model
        model2 = Model()
        model2.load_state_dict(copy.deepcopy(model.state_dict()))
        model2.cuda()
        model2.eval()
        y_pred_x, fc1, conv2, conv1 = model2(which_data[:,None,...].cuda()) # amplify zero sensitive ones
        
        if layer == 'conv1':
          avg_activation = torch.mean(conv1, dim=0).mean(-1).mean(-1)
          aa = avg_activation/avg_activation.max()          
          model2.conv1.bias -= 0.1*k*aa # * model2.fc1.bias  #*avg_activation
#           l_legend = [str(k*i) for i in range(0,11)]          
          l_legend = ['{0:.0f}'.format(0.1*k*i) for i in range(0,11)]
        elif  layer == 'conv2':
          avg_activation = torch.mean(conv2, dim=0).mean(-1).mean(-1)
          aa = avg_activation/avg_activation.max()          
          model2.conv2.bias -= k*aa # * model2.fc1.bias  #*avg_activation
          l_legend = [str(k*i) for i in range(0,11)]          
        else:
          avg_activation = torch.mean(fc1, dim=0)
          aa = avg_activation/avg_activation.max()
          model2.fc1.bias -= 0.01*k* avg_activation  #*avg_activation
          l_legend = ['{0:.1f}'.format(k*0.01*i) for i in range(0,11)]

        for idx, weight in enumerate(np.linspace(0,1,21)):    
            uu = test_x[test_y==which].clone()/255.
            z = torch.rand(uu.size(0), 28, 28) #.cuda()  
            z.cuda()

    #         stimuli = (1*z + weight*uu)/ (weight+1)
            stimuli = (1-weight)*z + weight*uu
#             stimuli = uu

            output, _, _, _ = model(stimuli[:,None,...].cuda())
            pred_n = output.data.max(1)[1]
            pred_n = pred_n.type(torch.ByteTensor).squeeze()

            if k==0:  
              axarr2[idx].imshow(stimuli[0])
              axarr2[idx].axis('off')      
              axarr2[idx].set_title('{:.2f}'.format((weight)))

            output_m, _, _, _ = model2(stimuli[:,None,...].cuda())
            pred_m = output_m.data.max(1)[1]
            pred_m = pred_m.type(torch.ByteTensor).squeeze()

            
            pp = (pred_n == which).sum().type(torch.FloatTensor) / pred_n.size(0) #       (test_y == which).sum()
            qq = (pred_m == which).sum().type(torch.FloatTensor)  / pred_m.size(0) #       (test_y == which).sum()  
            fraction_pred_favor.append([pp, qq])

        mat = np.array(fraction_pred_favor)
#         if k_idx == 0:
#           axarr[which].plot(mat[:,0], 'b')
        axarr[l_idx, which-5].plot(mat[:,1], color=(colors[k_idx,:3]))
        axarr[l_idx, which-5].set_yticks([0.5,1])     
#         if which==0:
        axarr[l_idx, which-5].set_ylabel('acc')
        axarr[l_idx, which-5].set_title(dig_names[which-5] + '-' + layer)
  
        axarr[l_idx, which-5].set_xticks([i for i in range(0,21,4)])       
        axarr[l_idx, which-5].set_xticklabels(['{:.1f}'.format(i/20) for i in range(0,21,4)])
        l_legend = ['- ' + q for q in l_legend]
        axarr[l_idx, which-5].legend(l_legend)                   

f.subplots_adjust(hspace=0.8) #, wspace=0.0, right = 0.8)
f.show()